In [4]:
import pandas as pd
import glob
import os
import tqdm
import gc
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import math
from sklearn import datasets, linear_model, model_selection
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from tqdm import tqdm

In [5]:
FILEPATH ='C:/Users/yuliya.tetereva/python/athletes_sochi.txt'
dfs = pd.read_csv(FILEPATH).dropna()

In [6]:
dfs

,age,birthdate,gender,height,name,weight,gold_medals,silver_medals,bronze_medals,total_medals,sport,country
0,17,1996-04-12,Male,1.72,Aaron Blunck,68.0,0,0,0,0,Freestyle Skiing,United States
1,27,1986-05-14,Male,1.85,Aaron March,85.0,0,0,0,0,Snowboard,Italy
2,21,1992-06-30,Male,1.78,Abzal Azhgaliyev,68.0,0,0,0,0,Short Track,Kazakhstan
4,21,1992-07-30,Male,1.86,Adam Barwood,82.0,0,0,0,0,Alpine Skiing,New Zealand
5,21,1992-12-18,Male,1.75,Adam Cieslar,57.0,0,0,0,0,Nordic Combined,Poland
...,...,...,...,...,...,...,...,...,...,...,...,...
2851,28,1985-04-30,Male,1.93,Ziga Pavlin,98.0,0,0,0,0,Ice Hockey,Slovenia
2853,31,1982-12-05,Female,1.70,Zina Kocher,60.0,0,0,0,0,Biathlon,Canada
2854,28,1985-06-14,Female,1.68,Zoe Gillings,65.0,0,0,0,0,Snowboard,Great Britain
2856,22,1991-03-01,Male,1.76,Zongyang Jia,68.0,0,0,1,1,Freestyle Skiing,China


In [7]:
def prepare_data (dfs, list_param):
    alldata = dfs[ list_param ]
    alldata = alldata.sample(frac=1)
    return alldata

alldata = prepare_data (dfs , ['weight', 'height', 'age','total_medals', 'gender'])
alldata

,weight,height,age,total_medals,gender
1477,40.0,1.72,21,0,Female
2474,77.0,1.85,28,0,Male
141,62.0,1.68,33,0,Male
237,74.0,1.74,22,0,Female
575,92.0,1.82,41,0,Male
...,...,...,...,...,...
2640,85.0,1.80,39,0,Male
1920,50.0,1.60,28,1,Female
21,73.0,1.75,20,0,Male
1669,52.0,1.62,28,0,Female


In [9]:
allinput = alldata[ ['weight', 'height', 'age','total_medals'] ]
alloutput = alldata["gender"].apply(lambda x: int(x == "Male"))

In [25]:
def prepare_test (a , alldata, allinput, alloutput):
    chunk_80 = int(len(alldata) * a)

    X_train = allinput[ :chunk_80]
    Y_train = alloutput[ :chunk_80]

    X_test = allinput[chunk_80: ]
    Y_test = alloutput[chunk_80: ]
    return X_train, Y_train, X_test, Y_test

X_train, Y_train, X_test_ , Y_test = prepare_test (0.8 , alldata, allinput, alloutput)
X_test_

,weight,height,age,total_medals
116,80.0,1.79,29,0
98,106.0,1.92,31,0
2530,67.0,1.70,20,0
157,56.0,1.68,26,0
1908,68.0,1.64,23,0
...,...,...,...,...
2640,85.0,1.80,39,0
1920,50.0,1.60,28,1
21,73.0,1.75,20,0
1669,52.0,1.62,28,0


In [21]:
Y_test

116     1
98      1
2530    0
157     0
1908    0
       ..
2640    1
1920    0
21      1
1669    0
2307    1
Name: gender, Length: 496, dtype: int64

In [26]:
def redr_model (X_train, Y_train, X_test):
    model = linear_model.LogisticRegression()
    model.fit(X_train, Y_train)
    Y_test_predicted = model.predict(X_test)
    return Y_test_predicted

Y_test_predicted = redr_model (X_train, Y_train, X_test_)
Y_test_predicted

array([1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,

In [30]:
def visual_resut (Y_test, Y_test_predicted):
    [tn, fp], [fn, tp] = confusion_matrix(Y_test, Y_test_predicted)
    print ('Confusion_matrix\n', confusion_matrix(Y_test, Y_test_predicted))
    precision = tp / (tp + fp)
    recall = tp / tp + fn

    print("Precision: ", tp / (tp + fp))
    print("Recall: ", tp / (tp + fn))
    print("Accuracy: ", (tn + tp) / (tp + fp + tn + fn))
    print("F1: ",  (precision * recall) / ((precision) + recall))
    
visual_resut (Y_test, Y_test_predicted)

Confusion_matrix
 [[131  32]
 [ 43 290]]
Precision:  0.9006211180124224
Recall:  0.8708708708708709
Accuracy:  0.8487903225806451
F1:  0.8825563701756812
